# Chapter 3. Dataframe Transformation

In [1]:
using CSV, DataFrames, RDatasets, Statistics

First, we load the iris data again to illustrate how to manipulate a dataframe in Julia:

In [2]:
df = dataset("datasets", "iris")
first(df, 5)

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa


## 1. Select a Subset (review)
In the last chapter, we knew how to select a subset. Now, let's do it again.  
In this example, we do not need `SepalWidth` column. For this purpose, we can either use:

In [3]:
df[:, [:SepalLength, :SepalWidth, :PetalLength, :PetalWidth]]

Row,SepalLength,SepalWidth,PetalLength,PetalWidth
,Float64,Float64,Float64,Float64
1,5.1,3.5,1.4,0.2
2,4.9,3.0,1.4,0.2
3,4.7,3.2,1.3,0.2
4,4.6,3.1,1.5,0.2
5,5.0,3.6,1.4,0.2
6,5.4,3.9,1.7,0.4
7,4.6,3.4,1.4,0.3
8,5.0,3.4,1.5,0.2
9,4.4,2.9,1.4,0.2


Or a logical calculation based selection:

In [4]:
df[:, Not(:SepalWidth)]

Row,SepalLength,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Cat…
1,5.1,1.4,0.2,setosa
2,4.9,1.4,0.2,setosa
3,4.7,1.3,0.2,setosa
4,4.6,1.5,0.2,setosa
5,5.0,1.4,0.2,setosa
6,5.4,1.7,0.4,setosa
7,4.6,1.4,0.3,setosa
8,5.0,1.5,0.2,setosa
9,4.4,1.4,0.2,setosa


## 2. Splition and Combination

### By Row
In R, you can use `rbind()` to combine two dataframes by appending new rows to the origional dataframe. Similarly in Julia, you can:

In [ ]:
df_a = DataFrame(col_a=[1, 2, 3], col_b=[4, 5, 6])
df_b = DataFrame(col_a=[2, 3, 3], col_b=[2, 3, 4])
df_bind = [df_a; df_b]
df_bind

Row,col_a,col_b
,Int64,Int64
1,1,4
2,2,5
3,3,6
4,2,2
5,3,3
6,3,4


Or, because the concatenation is vertical:

In [6]:
df_bind = vcat(df_a, df_b) # vertical, concat...
df_bind

Row,col_a,col_b
,Int64,Int64
1,1,4
2,2,5
3,3,6
4,2,2
5,3,3
6,3,4


However, if you want to make it faster on large datasets, `append!` is more efficient:

In [7]:
df_bind = append!(df_a, df_b)
df_bind

Row,col_a,col_b
,Int64,Int64
1,1,4
2,2,5
3,3,6
4,2,2
5,3,3
6,3,4


### By Column

**1. Bind horizontally**  Similar to the vertical concatenation, `vcat()`; we can use `hcat()` for horizontal concatenation:

In [ ]:
df_a = DataFrame(col_a=[1, 2, 3], col_b=[4, 5, 6])
df_b = DataFrame(col_c=[2, 3, 3], col_d=[2, 3, 4])
df_bind = hcat(df_a, df_b) # horizontal, concat...
df_bind

Row,col_a,col_b,col_c,col_d
,Int64,Int64,Int64,Int64
1,1,4,2,2
2,2,5,3,3
3,3,6,3,4


**2. Merge/join horizontally**

While concatenation is convenient, sometime, we have missing values and want to *safely* concat dataframes horizontally by `id` or values of a certain column. In any data pipelinee, there are `join` operations:

(1) Only id(s) existing in both dataframes

In [ ]:
df_a = DataFrame(id=[1, 2, 3], col_a=[4, 5, 6])
df_b = DataFrame(id=[2, 3, 3], col_b=[2, 3, 4])
df_bind = innerjoin(df_a, df_b, on=:id)
df_bind

Row,id,col_a,col_b
,Int64,Int64,Int64
1,2,5,2
2,3,6,3
3,3,6,4


(2) All id(s) existing in either dataframe

In [ ]:
df_bind = outerjoin(df_a, df_b, on=:id)
df_bind

Row,id,col_a,col_b
,Int64,Int64?,Int64?
1,2,5,2
2,3,6,3
3,3,6,4
4,1,4,missing


(3) Only id(s) existing in the dataframe on the left

In [ ]:
df_bind = leftjoin(df_a, df_b, on=:id)
df_bind

Row,id,col_a,col_b
,Int64,Int64,Int64?
1,2,5,2
2,3,6,3
3,3,6,4
4,1,4,missing


(4) Only id(s) existing in the dataframe on the right

In [ ]:
df_bind = rightjoin(df_a, df_b, on=:id)
df_bind

Row,id,col_a,col_b
,Int64,Int64?,Int64
1,2,5,2
2,3,6,3
3,3,6,4


## 3. Group by

To see the dataset by group, or to prepare for a sub-group calculation, we use

In [13]:
gdf = groupby(df, :Species)
first(gdf)

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


Based on this grouped dataframe (`gdf`), we can calculate the mean values of `SepalLength`:

In [14]:
combine(gdf, :SepalLength => mean)

Row,Species,SepalLength_mean
,Cat…,Float64
1,setosa,5.006
2,versicolor,5.936
3,virginica,6.588


What if we wanna know more than one columns?

In [15]:
combine(groupby(df, :Species), [:SepalLength, :SepalWidth] .=> mean)

Row,Species,SepalLength_mean,SepalWidth_mean
,Cat…,Float64,Float64
1,setosa,5.006,3.428
2,versicolor,5.936,2.77
3,virginica,6.588,2.974


## 4. Sorting

In many cases, after calculating the means of groups, we want to see which one is the highest. One way to finish the task is to sort the result:

In [ ]:
sort(combine(groupby(df, :Species), [:SepalLength, :SepalWidth] .=> mean),
    :SepalWidth_mean) # sort by SepalWidth_mean


Row,Species,SepalLength_mean,SepalWidth_mean
,Cat…,Float64,Float64
1,versicolor,5.936,2.77
2,virginica,6.588,2.974
3,setosa,5.006,3.428


Reversely:

In [ ]:
sort(combine(groupby(df, :Species), [:SepalLength, :SepalWidth] .=> mean),
    :SepalWidth_mean,
    rev=true) # sort by SepalWidth_mean


Row,Species,SepalLength_mean,SepalWidth_mean
,Cat…,Float64,Float64
1,setosa,5.006,3.428
2,virginica,6.588,2.974
3,versicolor,5.936,2.77


## 5. Transforming between long and wide tables

**(1) From a wide table to a long one:**

Let's look at the iris dataset again

In [18]:
first(df, 3)

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa


Assign an id column:

In [19]:
df.id = 1:size(df, 1)
first(df, 3)

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species,id
,Float64,Float64,Float64,Float64,Cat…,Int64
1,5.1,3.5,1.4,0.2,setosa,1
2,4.9,3.0,1.4,0.2,setosa,2
3,4.7,3.2,1.3,0.2,setosa,3


In [ ]:
df_long = stack(df, [:SepalLength, :SepalWidth, :PetalLength, :PetalWidth], variable_name=:variable, value_name=:value)
df_long


Row,Species,id,variable,value
,Cat…,Int64,String,Float64
1,setosa,1,SepalLength,5.1
2,setosa,2,SepalLength,4.9
3,setosa,3,SepalLength,4.7
4,setosa,4,SepalLength,4.6
5,setosa,5,SepalLength,5.0
6,setosa,6,SepalLength,5.4
7,setosa,7,SepalLength,4.6
8,setosa,8,SepalLength,5.0
9,setosa,9,SepalLength,4.4


**(2) From a long table to a wide one:**

In [21]:
df_wide = unstack(df_long, [:id, :Species], :variable, :value)
first(df_wide, 3)

Row,id,Species,SepalLength,SepalWidth,PetalLength,PetalWidth
,Int64,Cat…,Float64?,Float64?,Float64?,Float64?
1,1,setosa,5.1,3.5,1.4,0.2
2,2,setosa,4.9,3.0,1.4,0.2
3,3,setosa,4.7,3.2,1.3,0.2


Basically, the same to the original `df`:

In [22]:
first(df, 3)

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species,id
,Float64,Float64,Float64,Float64,Cat…,Int64
1,5.1,3.5,1.4,0.2,setosa,1
2,4.9,3.0,1.4,0.2,setosa,2
3,4.7,3.2,1.3,0.2,setosa,3


## 6. Missing values

Another important issue in data analysis is how to find and fill NA values.

In [23]:
df_bind = outerjoin(df_a, df_b, on=:id)
df_bind

Row,id,col_a,col_b
,Int64,Int64?,Int64?
1,2,5,2
2,3,6,3
3,3,6,4
4,1,4,missing


### Fina NA values

In [25]:
filter(:col_b => ismissing, df_bind)

Row,id,col_a,col_b
,Int64,Int64?,Int64?
1,1,4,missing


### Fill NA values

In [28]:
coalesce.(df_bind, 0)

Row,id,col_a,col_b
,Int64,Int64,Int64
1,2,5,2
2,3,6,3
3,3,6,4
4,1,4,0


Please pay attention to the dot, `.` after `coalesce`, which means that we are casting the function, `coalesce`, to each element (**cell**) within the dataframe.

### Drop NA values

In [26]:
dropmissing(df_bind, :col_b)

Row,id,col_a,col_b
,Int64,Int64?,Int64
1,2,5,2
2,3,6,3
3,3,6,4


For how to simplift the process in a long pipeline, it's time to [enter the next chapter](https://reynards-org.gitbook.io/data-analysis-in-julia/4.pipeline.tools.jl).